# Preliminary Stuff

## Overview

Make the world a safer place by identifying distracted drivers ✌
- Using VGG16 pretrained network
- Data set : https://www.kaggle.com/c/state-farm-distracted-driver-detection

## Prerequisites

### Download + Unzip Data

- StateFarm Data Set : https://www.kaggle.com/c/state-farm-distracted-driver-detection
- Place unzipped contents in ./data/ folder relative to this notebook's current folder

### Copy a small sample of the data for testing

Copy first 10 images from each category to a sample folder with same structure

In [20]:
import shutil
import os

trainRoot, testRoot, sampleRoot = 'data/train/', 'data/test/', 'data/sample/'
numFiles = 10

def copyNumFiles(srcFolder, destFolder, numFiles):
    files = os.listdir(srcFolder)
    for file in files[0:numFiles]:
        src = srcFolder+'/'+file
        dest = destFolder+'/'+file
        
        if not(os.path.isdir(destFolder)):
            os.makedirs(destFolder)
        
        shutil.copy2(src, dest)
        print(src, ' -> ', dest)

# Copy Training Data
trainFolders = os.listdir(trainRoot)
for cur in trainFolders:
    copyNumFiles(trainRoot + cur, sampleRoot+'train/'+cur, numFiles)

copyNumFiles(testRoot, sampleRoot+'test/', numFiles)

data/train/c0/img_100026.jpg  ->  data/sample/train/c0/img_100026.jpg
data/train/c0/img_10003.jpg  ->  data/sample/train/c0/img_10003.jpg
data/train/c0/img_100050.jpg  ->  data/sample/train/c0/img_100050.jpg
data/train/c0/img_100074.jpg  ->  data/sample/train/c0/img_100074.jpg
data/train/c0/img_10012.jpg  ->  data/sample/train/c0/img_10012.jpg
data/train/c0/img_100145.jpg  ->  data/sample/train/c0/img_100145.jpg
data/train/c0/img_100191.jpg  ->  data/sample/train/c0/img_100191.jpg
data/train/c0/img_100257.jpg  ->  data/sample/train/c0/img_100257.jpg
data/train/c0/img_100312.jpg  ->  data/sample/train/c0/img_100312.jpg
data/train/c0/img_100337.jpg  ->  data/sample/train/c0/img_100337.jpg
data/train/c1/img_100021.jpg  ->  data/sample/train/c1/img_100021.jpg
data/train/c1/img_100045.jpg  ->  data/sample/train/c1/img_100045.jpg
data/train/c1/img_100046.jpg  ->  data/sample/train/c1/img_100046.jpg
data/train/c1/img_10011.jpg  ->  data/sample/train/c1/img_10011.jpg
data/train/c1/img_100126.j

Allows plots to be displayed right in the jupyter notebook

In [1]:
%matplotlib inline

Use path variable to easily switch between sub-sample (for speed) and full dataset

In [2]:
#path = 'data/sample'
path = '/media/angus/Windows/Users/mcleana/workspace/machine-learning/FastAI Course/Lesson 1/data/sample/'

# Begining

## Imports

Import / Initialize Libraries

In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

Set evironment vars for Theano to run without GPU backend

In [2]:
os.environ["THEANO_FLAGS"] = "device=cuda"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["MKL_THREADING_LAYER"] = "GNU"

Import helper files from FastAI

In [4]:
import utils
from utils import plots

Using Theano backend.


In [5]:
# Import our class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

## 

Set the batch size

In [6]:
batch_size=64

# VGG16 Learning

In [7]:
vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)

# fixes for keras 2 vs keras 1
batches.nb_class = batches.num_class
batches.nb_sample = batches.samples
val_batches.nb_class = batches.num_class
val_batches.nb_sample = val_batches.samples

print('Starting training...')
vgg.finetune(batches)
print('Done finetune, starting .fit()')
vgg.fit(batches, val_batches, nb_epoch=1)
print('Done .fit()')

vgg16.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
vgg16.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
vgg16.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
vgg16.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))


Found 100 images belonging to 10 classes.
Found 100 images belonging to 10 classes.
Starting training...
Done finetune, starting .fit()


vgg16.py:213: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
vgg16.py:213: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=1, epochs=1, validation_steps=100)`
  validation_data=val_batches, nb_val_samples=val_batches.nb_sample)


Epoch 1/1


KeyboardInterrupt: 

Now use model to predict images in test set

In [ ]:
test_batches = vgg.get_batches(path+'valid', batch_size=4)
imgs,labels = next(test_batches)
plots(imgs, titles=labels)